In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc

In [2]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_features_matrixregV4_norm.csv'
day_list = ['day7', 'day8', 'day9']
df_dict = {}
for file in ['day7', 'day8', 'day9','test']: 
    df_dict[file] = pd.read_csv(load_path+file_format.format(file))
    print(file_format.format(file))

day7_features_matrixregV4_norm.csv
day8_features_matrixregV4_norm.csv
day9_features_matrixregV4_norm.csv
test_features_matrixregV4_norm.csv


In [3]:
## load category col
file_format = '{}_categorymore_V1.csv'
df_dict_cat = {}
for file in ['day7', 'day8', 'day9','test']: 
    df_dict_cat[file] = pd.read_csv(load_path+file_format.format(file))
    print(file_format.format(file))

day7_categorymore_V1.csv
day8_categorymore_V1.csv
day9_categorymore_V1.csv
test_categorymore_V1.csv


In [5]:
# replace/add new discrete cols
for file in ['day7', 'day8', 'day9','test']: 
    for col in df_dict_cat['test'].columns:
        df_dict[file][col] = df_dict_cat[file][col]

In [33]:
df_dict['day7'].head(5)

,ip_day_hour_count,ip_os_day_hour_count,ip_app_day_hour_count,ip_app_os_day_hour_count,app_day_hour_count,ip_device_os_count,ip_app_device_os_count,ip_device_os_mean,ip_app_device_os_mean,ip_app_device_mean,...,channel,device,os,hour,is_attributed,app_device,app_os,device_os,app_device_os_channel,app_device_os
0,11,9,3,3,430196,31,11,0.000000,0.0,0.000000,...,245,1,17,4,0,367,2580,51,4515,4515
1,71,9,8,3,430196,121,21,0.013333,0.0,0.000000,...,245,1,18,4,0,367,2581,52,4516,4516
2,11,9,3,3,312628,31,9,0.000000,0.0,0.000000,...,317,1,17,4,0,410,3872,51,6652,6652
3,655,129,222,124,327851,1008,260,0.000000,0.0,0.000199,...,245,1,10,4,0,389,3394,44,5839,5839
4,55,55,11,11,754715,59,12,0.000000,0.0,0.000000,...,480,1,8,4,0,304,753,42,1447,1447


# define parameter

In [34]:
df_dict_cat['test'].columns

Index(['app_device', 'app_os', 'device_os', 'app_device_os_channel',
       'app_device_os'],
      dtype='object')

In [35]:
target = 'is_attributed'

day_list = ['day7', 'day8', 'day9']

combine = 0
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 61,
        'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':70,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        'verbose': 0,
        'scale_pos_weight': 400,
        'metric' : [ 'auc']
    }
categorical_col = [ 'app', 'device', 'os', 'channel', 'hour',   ]

feature_cols = list(df_dict['test'].columns.values)

In [36]:
df_dict['day7'].columns.values

array(['ip_day_hour_count', 'ip_os_day_hour_count',
       'ip_app_day_hour_count', 'ip_app_os_day_hour_count',
       'app_day_hour_count', 'ip_device_os_count',
       'ip_app_device_os_count', 'ip_device_os_mean',
       'ip_app_device_os_mean', 'ip_app_device_mean', 'app_device_os_mean',
       'ip_device_os_time2nextclick', 'ip_app_device_os_time2nextclick',
       'ip_app_device_time2nextclick', 'app_device_os_time2nextclick',
       'ip_device_os_time2previousclick',
       'ip_app_device_os_time2previousclick',
       'ip_app_device_time2previousclick',
       'app_device_os_time2previousclick', 'ip_device_os_countfromfuture',
       'ip_app_device_os_countfromfuture', 'ip_app_device_countfromfuture',
       'app_device_os_countfromfuture', 'ip_device_os_countfrompast',
       'ip_app_device_os_countfrompast', 'ip_app_device_countfrompast',
       'app_device_os_countfrompast', 'ip_device_os_lasttimediff',
       'ip_app_device_os_lasttimediff', 'ip_app_device_lasttimediff',
  

In [37]:
# get rid of extra features
# extra = set(['matrixFact_user_iposdeviceapp_item_channel', 'matrixFact_user_iposdeviceapp_item_device'])
extra = set(['app_device_os_mean',
             'app_device_os_time2nextclick', 
             'app_device_os_time2previousclick',
             'app_device_os_countfromfuture',
             'app_device_os_countfrompast', 
             'app_device_os_lasttimediff', 
             'app_device_os_regression', 'app_device_os_channel', 'app_device_os', 'app_os', 'app_device', 'device_os',
             'ip_app_device_mean', 'ip_device_os_mean', 'ip_app_device_os_mean'])
# extra = set(['ip_app_device_mean',
#              'ip_app_device_time2nextclick', 
#              'ip_app_device_time2previousclick',
#              'ip_app_device_countfromfuture',
#              'ip_app_device_countfrompast', 
#              'ip_app_device_lasttimediff', 
#              'ip_app_device_regression'])
feature_cols = list(set(feature_cols) - extra)

In [39]:
feature_cols

['os',
 'ip_app_device_os_time2previousclick',
 'matrixFact_user_iposdeviceapp_item_app',
 'ip_app_device_time2nextclick',
 'ip_app_device_os_time2nextclick',
 'ip_app_device_os_countfrompast',
 'ip_app_device_countfromfuture',
 'ip_app_device_os_lasttimediff',
 'app_day_hour_count',
 'ip_device_os_count',
 'ip_app_day_hour_count',
 'ip_app_device_time2previousclick',
 'ip_app_device_os_count',
 'ip_device_os_time2nextclick',
 'ip_device_os_countfrompast',
 'device',
 'ip_device_os_countfromfuture',
 'ip_app_device_os_countfromfuture',
 'app',
 'matrixFact_user_ip_item_appdeviceos',
 'ip_app_device_countfrompast',
 'ip_device_os_time2previousclick',
 'ip_app_device_lasttimediff',
 'ip_device_os_regression',
 'ip_app_device_regression',
 'ip_app_device_os_regression',
 'ip_device_os_lasttimediff',
 'hour',
 'ip_app_os_day_hour_count',
 'channel',
 'ip_os_day_hour_count',
 'ip_day_hour_count']

In [40]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
    if best_round is not None:
        param['num_rounds'] = best_round
        del param['early_stopping_round']
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model

In [ ]:
for day in day_list:
    train_day = list(set(day_list)-set([day]))
    file_name = 'concat_{}{}_val_{}_predtest_matrixregV4_norm_categorymore_V1_nomean.npy'.format(train_day[0],train_day[1],day)
    print(file_name)
    trainset = pd.concat([df_dict[train_day[0]],df_dict[train_day[1]]])
    valset = df_dict[day]
    print('building train val done!')
    model = train_lightgbm(trainset, valset, feature_cols, categorical_col, params)
    best_round = model.best_iteration
    df_all = pd.concat([trainset, valset])
    model = train_lightgbm(df_all, valset, feature_cols, categorical_col, params, best_round)
    del df_all
    gc.collect()
    pred = model.predict(df_dict['test'][feature_cols])
    np.save(load_path+file_name, pred)
    
    # prediction
    df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
    print('loading file done!')
    df_sub = pd.DataFrame()
    df_sub['click_id'] = df_test_raw['click_id']
    df_sub['is_attributed'] = pred
    print('predicting file done!')
    df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_{}{}_val_{}_matrixregV4_norm_categorymore_V1_nomean.csv.gz'.format(train_day[0],train_day[1],day), compression='gzip', index=False)

    

concat_day8day9_val_day7_predtest_matrixregV4_norm_categorymore_V1_nomean.npy
building train val done!
start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.979804
[20]	valid_0's auc: 0.981563
[30]	valid_0's auc: 0.982393
[40]	valid_0's auc: 0.983051
[50]	valid_0's auc: 0.983665
[60]	valid_0's auc: 0.984013
[70]	valid_0's auc: 0.984132
[80]	valid_0's auc: 0.984194
[90]	valid_0's auc: 0.984295
[100]	valid_0's auc: 0.984335
[110]	valid_0's auc: 0.984372
[120]	valid_0's auc: 0.98433
[130]	valid_0's auc: 0.984345
[140]	valid_0's auc: 0.984338
[150]	valid_0's auc: 0.984333
[160]	valid_0's auc: 0.984333
[170]	valid_0's auc: 0.984318
[180]	valid_0's auc: 0.984279
[190]	valid_0's auc: 0.984257
[200]	valid_0's auc: 0.984236
[210]	valid_0's auc: 0.984233
Early stopping, best iteration is:
[110]	valid_0's auc: 0.984372
start training
[10]	valid_0's auc: 0.981062
[20]	valid_0's auc: 0.983115
[30]	valid_0's auc: 0.984265
[40]	valid_0's auc: 0.985482
[50]	valid_0's auc: 0.986437
[60]	valid_0's auc: 0.987088
[70]	valid_0's auc: 0.987619
[80]	valid_0's auc: 0.988282
[90]	

In [32]:
# importance_type (string, optional (default="split")) 
# – How the importance is calculated. 
# If “split”, result contains numbers of times the feature is used in a model. 
# If “gain”, result contains total gains of splits which use the feature.
importance = pd.Series(model.feature_importance(importance_type='gain'), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
    
importance

matrixFact_user_ip_item_appdeviceos       3.446741e+08
matrixFact_user_iposdeviceapp_item_app    8.417493e+07
channel                                   1.668758e+07
device_os                                 1.108526e+07
ip_app_device_os_time2nextclick           8.144030e+06
app                                       6.917718e+06
ip_app_device_os_lasttimediff             6.707789e+06
ip_day_hour_count                         4.397688e+06
ip_app_device_os_countfromfuture          3.394890e+06
os                                        2.623705e+06
ip_device_os_count                        2.545699e+06
ip_app_os_day_hour_count                  2.149142e+06
hour                                      1.961198e+06
ip_app_device_time2nextclick              1.530539e+06
ip_device_os_lasttimediff                 1.250323e+06
ip_app_device_mean                        8.831188e+05
ip_app_device_regression                  8.715329e+05
ip_app_device_lasttimediff                8.461638e+05
ip_device_

In [ ]:
importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
    
importance

# Average

In [42]:
#### load each and average
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
day7 = np.load(load_path+'concat_day8day9_val_day7_predtest_matrixregV4_norm_categorymore_V1_nomean.npy')
day8 = np.load(load_path+'concat_day7day9_val_day8_predtest_matrixregV4_norm_categorymore_V1_nomean.npy')
day9 = np.load(load_path+'concat_day7day8_val_day9_predtest_matrixregV4_norm_categorymore_V1_nomean.npy')

pred = (day7 + day8 + day9)/3

In [43]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_blend_matrixregV4_norm_categorymore_V1_nomean.csv.gz', compression='gzip', index=False)


loading file done!
predicting file done!


In [ ]:
'/home/kai/data/kaggle/talkingdata/wl/data/equalhour/concat_day8day9_val_day7_predtest_matrixregV2_exclude_leaf61_compensate.npy'